<a href="https://colab.research.google.com/github/ZeyTrack/CodSoft/blob/main/MOVIE_RATING_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CODSOFT INTERNSHIP**

TASK - 2 : MOVIE RATING PREDICTION

Author : Zinab

Domain : Data Science

Batch : july


## 1. Imports and Reading The *Data*

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score



In [120]:
!pip install ydata_profiling

In [121]:
original_data = pd.read_csv('/content/drive/MyDrive/CodeSoft/movies_data/IMDb Movies India.csv', encoding='latin-1')
data = original_data.copy()
data

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali
...,...,...,...,...,...,...,...,...,...,...
15504,Zulm Ko Jala Doonga,(1988),NaN,Action,4.6,11,Mahendra Shah,Naseeruddin Shah,Sumeet Saigal,Suparna Anand
15505,Zulmi,(1999),129 min,"Action, Drama",4.5,655,Kuku Kohli,Akshay Kumar,Twinkle Khanna,Aruna Irani
15506,Zulmi Raj,(2005),NaN,Action,NaN,NaN,Kiran Thej,Sangeeta Tiwari,NaN,NaN
15507,Zulmi Shikari,(1988),NaN,Action,NaN,NaN,NaN,NaN,NaN,NaN


##2. Explore and Clean Data

In [122]:
import ydata_profiling

In [123]:
report=ydata_profiling.ProfileReport(data, minimal=True)
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [124]:
data.describe()

,Rating
count,7919.000000
mean,5.841621
std,1.381777
min,1.100000
25%,4.900000
50%,6.000000
75%,6.800000
max,10.000000


In [125]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15509 entries, 0 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      15509 non-null  object 
 1   Year      14981 non-null  object 
 2   Duration  7240 non-null   object 
 3   Genre     13632 non-null  object 
 4   Rating    7919 non-null   float64
 5   Votes     7920 non-null   object 
 6   Director  14984 non-null  object 
 7   Actor 1   13892 non-null  object 
 8   Actor 2   13125 non-null  object 
 9   Actor 3   12365 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.2+ MB


In [126]:
data.isna().sum()

,0
Name,0
Year,528
Duration,8269
Genre,1877
Rating,7590
Votes,7589
Director,525
Actor 1,1617
Actor 2,2384
Actor 3,3144


In [127]:
data.shape

(15509, 10)

In [128]:
data.duplicated().sum()

6

In [129]:
data.dropna(inplace=True)

In [130]:
data.shape

(5659, 10)

In [131]:
data.columns

Index(['Name', 'Year', 'Duration', 'Genre', 'Rating', 'Votes', 'Director',
       'Actor 1', 'Actor 2', 'Actor 3'],
      dtype='object')

In [132]:
data.isna().sum()

,0
Name,0
Year,0
Duration,0
Genre,0
Rating,0
Votes,0
Director,0
Actor 1,0
Actor 2,0
Actor 3,0


## 3.   Data Preprocessing


In [133]:
data.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
5,...Aur Pyaar Ho Gaya,(1997),147 min,"Comedy, Drama, Musical",4.7,827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor
6,...Yahaan,(2005),142 min,"Drama, Romance, War",7.4,"1,086",Shoojit Sircar,Jimmy Sheirgill,Minissha Lamba,Yashpal Sharma
8,?: A Question Mark,(2012),82 min,"Horror, Mystery, Thriller",5.6,326,Allyson Patel,Yash Dave,Muntazir Ahmad,Kiran Bhatia


In [134]:
data['Votes'] = pd.to_numeric(data['Votes'].astype(str).str.replace(',','').astype(int))
data['Year'] = data['Year'].astype(str).str.strip('()').astype(int)
data['Duration'] = pd.to_numeric(data['Duration'].astype(str).str.replace('min',''))
data['Genre'] = data['Genre'].str.split(', ')
data = data.explode('Genre')
data['Genre'].fillna(data['Genre'].mode()[0], inplace = True)

In [135]:
data.isna().sum()

,0
Name,0
Year,0
Duration,0
Genre,0
Rating,0
Votes,0
Director,0
Actor 1,0
Actor 2,0
Actor 3,0


In [136]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11979 entries, 1 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      11979 non-null  object 
 1   Year      11979 non-null  int64  
 2   Duration  11979 non-null  int64  
 3   Genre     11979 non-null  object 
 4   Rating    11979 non-null  float64
 5   Votes     11979 non-null  int64  
 6   Director  11979 non-null  object 
 7   Actor 1   11979 non-null  object 
 8   Actor 2   11979 non-null  object 
 9   Actor 3   11979 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 1.0+ MB


In [137]:
data.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
1,#Gadhvi (He thought he was Gandhi),2019,109,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
3,#Yaaram,2019,110,Comedy,4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
3,#Yaaram,2019,110,Romance,4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
5,...Aur Pyaar Ho Gaya,1997,147,Comedy,4.7,827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor
5,...Aur Pyaar Ho Gaya,1997,147,Drama,4.7,827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor


In [138]:
top_movie = data.loc[data['Rating'].sort_values(ascending=False)[:10].index]
top_movie

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
8339,Love Qubool Hai,2020,94,Drama,10.0,5,Saif Ali Sayeed,Ahaan Jha,Mahesh Narayan,Rajasree Rajakumari
8339,Love Qubool Hai,2020,94,Romance,10.0,5,Saif Ali Sayeed,Ahaan Jha,Mahesh Narayan,Rajasree Rajakumari
8339,Love Qubool Hai,2020,94,Drama,10.0,5,Saif Ali Sayeed,Ahaan Jha,Mahesh Narayan,Rajasree Rajakumari
8339,Love Qubool Hai,2020,94,Romance,10.0,5,Saif Ali Sayeed,Ahaan Jha,Mahesh Narayan,Rajasree Rajakumari
5410,Half Songs,2021,79,Music,9.7,7,Sriram Raja,Raj Banerjee,Emon Chatterjee,Purshottam Mulani
5410,Half Songs,2021,79,Romance,9.7,7,Sriram Raja,Raj Banerjee,Emon Chatterjee,Purshottam Mulani
5410,Half Songs,2021,79,Music,9.7,7,Sriram Raja,Raj Banerjee,Emon Chatterjee,Purshottam Mulani
5410,Half Songs,2021,79,Romance,9.7,7,Sriram Raja,Raj Banerjee,Emon Chatterjee,Purshottam Mulani
5077,Gho Gho Rani,2019,105,History,9.4,47,Munni Pankaj,Nishi Neha Mishra,Pankaj Kamal,Akash Kumar
5077,Gho Gho Rani,2019,105,Romance,9.4,47,Munni Pankaj,Nishi Neha Mishra,Pankaj Kamal,Akash Kumar


In [139]:
data.shape

(11979, 10)

## 4.Data Visulization

In [140]:
import plotly.express as px

year = px.histogram(data, x = 'Year',histnorm = 'probability density', nbins = 20)
year

In [141]:
ratingg = px.histogram(data, x='Rating',histnorm = 'probability density', nbins = 30)
ratingg

In [142]:

data.drop('Name', axis = 1, inplace = True)



In [143]:
genre_m = data.groupby('Genre')['Rating'].transform('mean')
data['Genre_mean_rating'] = genre_m

director_m = data.groupby('Director')['Rating'].transform('mean')
data['Director_encoded'] = director_m

actor1_m = data.groupby('Actor 1')['Rating'].transform('mean')
data['Actor1_encoded'] = actor1_m

actor2_m = data.groupby('Actor 2')['Rating'].transform('mean')
data['Actor2_encoded'] = actor2_m

actor3_m = data.groupby('Actor 3')['Rating'].transform('mean')
data['Actor3_encoded'] = actor3_m

## 5.Splitting the Dataset

In [144]:
X = data[[ 'Year', 'Votes', 'Duration', 'Genre_mean_rating','Director_encoded','Actor1_encoded', 'Actor2_encoded', 'Actor3_encoded']]
y = data['Rating']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


## 6.Model Evaluation


In [145]:
Model = LinearRegression()
Model.fit(X_train,y_train)
prediction = Model.predict(X_test)

print('The performance: ', '\n')
print('RMSE: ',mean_squared_error(y_test, prediction))
print('MAE: ',mean_absolute_error(y_test, prediction))
print('R-squared: ',r2_score(y_test, prediction))

The performance:  

RMSE:  0.4465441653985704
MAE:  0.4921902540765641
R-squared:  0.7641133663863862


## 7. Testing

In [146]:
test = {'Year': [1997], 'Votes': [827], 'Duration': [147], 'Genre_mean_rating': [6.056744], 'Director_encoded': [5.335135], 'Actor1_encoded': [4.793617], 'Actor2_encoded': [5.73], 'Actor3_encoded': [5.93]}
tr = pd.DataFrame(test)

prediction = Model.predict(tr)
print("Predicted Rating:", prediction[0])


Predicted Rating: 5.4007172352130555
